In [5]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize
import pygsheets

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [6]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'api_key':config.api_key}

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()
comm_ids

['C00715532',
 'C00722108',
 'C00713958',
 'C00710087',
 'C00314575',
 'C00722454',
 'C00709899',
 'C00653816',
 'C00703413',
 'C00706739',
 'C00683193',
 'C00433391',
 'C00723411',
 'C00709279',
 'C00712323',
 'C00694364']

In [7]:
#Print committee list to validate
cand_list=json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']]
print(cand_list)

   committee_id                             name party
0     C00715532              JAY ALLEN FOR MAINE   REP
1     C00722108            ADRIENNE FOR CONGRESS   REP
2     C00713958              BRAKEY FOR CONGRESS   REP
3     C00710087         MICHAEL BUNKER FOR MAINE   DEM
4     C00314575              COLLINS FOR SENATOR   REP
5     C00722454         DALE CRAFTS FOR CONGRESS   REP
6     C00709899            SARA GIDEON FOR MAINE   DEM
7     C00653816  COMMITTEE TO ELECT JARED GOLDEN   DEM
8     C00703413            FRIENDS OF BRE KIDMAN   DEM
9     C00706739            DEREK FOR U.S. SENATE   REP
10    C00683193      DR CAT LONDON FOR US SENATE   DEM
11    C00433391             PINGREE FOR CONGRESS   DEM
12    C00723411                     LISAFORMAINE   UNK
13    C00709279                 SWEET FOR SENATE   DEM
14    C00712323            JON TREACY FOR SENATE   DEM
15    C00694364   DANIELLE VANHELSING FOR SENATE   IND


In [8]:
#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)
p=int(1)
    
#Loop through pages
for x in range(len(comm_ids)):
    
    #Print out candidate
    print(cand_list.iloc[id]['name'])
    
    querydict = {'per_page':'100'
                ,'api_key':config.api_key
                ,'committee_id':comm_ids[id]
                ,'page':p
                ,'sort_nulls_last':'false'
                ,'sort':'-contribution_receipt_date'
                ,'sort_hide_null':'false'
                }
    
    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/efile/',params=querydict).json()
    
    #Loop through pages
    while p <= r['pagination']['pages']:
        
        print('page: '+str(p)+','+str(r['pagination']['per_page'])+' records')
        
        querydict.update(page=p)
        #Pull new results
        r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
        
        #Add results to dataframe
        df = json_normalize(r['results'])
        dfs.append(df)
        p=p+1
    
    #Reset to page 1
    p=int(1)
    #Increment to next candidate
    id=id+1

print('Job Complete')

JAY ALLEN FOR MAINE
ADRIENNE FOR CONGRESS
BRAKEY FOR CONGRESS
page: 1,100 records
MICHAEL BUNKER FOR MAINE
COLLINS FOR SENATOR
page: 1,100 records
page: 2,100 records
page: 3,100 records
page: 4,100 records
page: 5,100 records
page: 6,100 records
page: 7,100 records
page: 8,100 records
page: 9,100 records
page: 10,100 records
page: 11,100 records
page: 12,100 records
page: 13,100 records
page: 14,100 records
page: 15,100 records
page: 16,100 records
page: 17,100 records
page: 18,100 records
page: 19,100 records
page: 20,100 records
page: 21,100 records
page: 22,100 records
page: 23,100 records
page: 24,100 records
page: 25,100 records
page: 26,100 records
page: 27,100 records
page: 28,100 records
page: 29,100 records
page: 30,100 records
page: 31,100 records
page: 32,100 records
page: 33,100 records
page: 34,100 records
page: 35,100 records
page: 36,100 records
page: 37,100 records
page: 38,100 records
page: 39,100 records
page: 40,100 records
page: 41,100 records
page: 42,100 records


page: 16,100 records
page: 17,100 records
page: 18,100 records
page: 19,100 records
page: 20,100 records
page: 21,100 records
page: 22,100 records
page: 23,100 records
page: 24,100 records
page: 25,100 records
page: 26,100 records
page: 27,100 records
page: 28,100 records
page: 29,100 records
page: 30,100 records
page: 31,100 records
page: 32,100 records
page: 33,100 records
page: 34,100 records
page: 35,100 records
page: 36,100 records
page: 37,100 records
page: 38,100 records
page: 39,100 records
page: 40,100 records
page: 41,100 records
page: 42,100 records
page: 43,100 records
page: 44,100 records
page: 45,100 records
page: 46,100 records
page: 47,100 records
page: 48,100 records
page: 49,100 records
page: 50,100 records
page: 51,100 records
page: 52,100 records
page: 53,100 records
page: 54,100 records
page: 55,100 records
page: 56,100 records
page: 57,100 records
page: 58,100 records
page: 59,100 records
page: 60,100 records
page: 61,100 records
page: 62,100 records
page: 63,100 

In [9]:
itemall=pd.concat(dfs,sort=False)
itemdf=itemall.drop_duplicates(subset='transaction_id')
itemdf

,amendment_indicator,amendment_indicator_desc,back_reference_schedule_name,back_reference_transaction_id,candidate_first_name,candidate_id,candidate_last_name,candidate_middle_name,candidate_name,candidate_office,...,recipient_committee_org_type,recipient_committee_type,report_type,report_year,schedule_type,schedule_type_full,sub_id,transaction_id,two_year_transaction_period,unused_contbr_id
0,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967060,A306338B7EF6B496CB90,2020,None
1,A,ADD,None,None,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967059,AC34F8C9B737C40F5955,2020,None
2,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967057,A70279EE102004960A9B,2020,None
3,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967056,A3229995F593145FBA35,2020,None
4,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967055,ACE18285784D042BE976,2020,None
5,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967054,A1BE9801B91B94045AC2,2020,None
6,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967053,A14E09540007B44E8AC6,2020,None
7,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967052,AB2E5F77495414A1DA97,2020,None
8,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967051,A4D900FCF49B04D0391D,2020,None
9,A,ADD,SA12,A2C91FC04AD224568880,None,None,None,None,None,None,...,None,S,Q3,2019,SA,ITEMIZED RECEIPTS,4102020191674967050,A7819DA91BFD84FEF868,2020,None


In [13]:
print(itemall.columns)

Index(['amendment_indicator', 'amendment_indicator_desc',
       'back_reference_schedule_name', 'back_reference_transaction_id',
       'candidate_first_name', 'candidate_id', 'candidate_last_name',
       'candidate_middle_name', 'candidate_name', 'candidate_office',
       ...
       'recipient_committee_org_type', 'recipient_committee_type',
       'report_type', 'report_year', 'schedule_type', 'schedule_type_full',
       'sub_id', 'transaction_id', 'two_year_transaction_period',
       'unused_contbr_id'],
      dtype='object', length=124)


In [15]:
#Itemized dataframe query
itemdf=itemdf[[
                'committee.city'
                ,'committee.committee_id'
                ,'committee.committee_type_full'
                ,'committee.cycle'
                ,'committee.name'
                ,'committee.party_full'
                ,'committee.state_full'
                ,'contribution_receipt_amount'
                ,'contribution_receipt_date'
                ,'contributor_aggregate_ytd'
                ,'contributor_city'
                ,'contributor_employer'
                ,'contributor_first_name'
                ,'contributor_last_name'
                ,'contributor_name'
#                 ,'cycle'
                ,'contributor_occupation'
                ,'contributor_state'
                ,'contributor_zip'
                ,'entity_type'
#                 ,'filing.coverage_end_date'
#                 ,'filing.coverage_start_date'
#                 ,'filing.filed_date'
#                 ,'filing.is_amended'
                ,'line_number'
                ,'fec_election_type_desc'
                ,'memo_text'
#                 ,'filing.pdf_url'
                ,'transaction_id']]
len(itemdf.index)

542

In [16]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/maine-fed-raw-itemized-receipts.csv')

In [ ]:
# #Write itemized individual raw results to Google Sheet
# cwd = os.getcwd()

# #Google Credentials
# gc = pygsheets.authorize(service_file=cwd+'/me-congress-2020-creds.json')

# #Select sheet and worksheet
# sh = gc.open('me-congress-2020')
# #sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
# wks = sh[3]

# #Clear sheet before load
# wks.clear(start='A1',fields='*')

# #Write contribs dataframe to sheet
# wks.set_dataframe(itemdf,(1,1))